In [ ]:
import pandas as pd 
import pickle
import datetime

#  LVEF 

In [ ]:
df = pd.read_csv('/projects/data/GS/HF/from_jack/FULL_TEXT_cardiac_echo_report_v2_with_vars.csv')
del df['Unnamed: 0']

In [ ]:
df.info()

#  HF Patients on spironolactone  

In [ ]:
with open('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/first_mention_spiro.pickle', 'rb') as f:
    first_mention_spiro = pickle.load(f)
len(first_mention_spiro)

In [ ]:
first_mention_spiro = pd.DataFrame(list(first_mention_spiro.items()), columns=['client_idcode', 'first_mention_spiro'])
first_mention_spiro.info()

#  Extracting LVEF data for each HF patient on spironolactone

In [ ]:
df = pd.merge(df, first_mention_spiro, on='client_idcode', how='right')
df = df[df['ef_value'].notna()]

In [ ]:
len(df.loc[df['document_datecreated'].isna()])

In [ ]:
df['document_datecreated'] = pd.to_datetime(df['document_datecreated'], utc=True).dt.date 
df['difference'] = pd.to_datetime(df['first_mention_spiro']) - pd.to_datetime(df['document_datecreated'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df = df.loc[df['in_window'] == True]
df.shape

In [ ]:
df.head()

In [ ]:
duplicates = df['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
df = df[df.groupby('client_idcode').document_datecreated.transform('max') == df['document_datecreated']]
df.shape

In [ ]:
df['result'] = df.groupby('client_idcode')['ef_value'].transform('mean')
df.drop_duplicates(subset='client_idcode', keep='first', inplace = True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['result'].isna().value_counts()

In [ ]:
lvef_dict = dict(zip(df.client_idcode, df.result))
len(lvef_dict)

In [ ]:
with open ('/projects/RALES TRIAL/1A/spiro dataextraction/spiro data/lvef_dict.pickle', 'wb') as f:
    pickle.dump(lvef_dict,f)